In [16]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tox21data/tox21.csv


In [17]:
!pip install torch-geometric
!pip install rdkit-pypi


In [18]:
import torch
import torch.nn as nn
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GATConv, global_mean_pool, global_max_pool, global_add_pool
from torch_geometric.explain import Explainer, GNNExplainer
from rdkit import Chem
from rdkit.Chem import Descriptors
import numpy as np


## Preprocessing

In [19]:
import pandas as pd
import numpy as np
from rdkit import Chem
import torch
from torch_geometric.data import Data
import random

# ============================================================================
# LOAD DỮ LIỆU
# ============================================================================

TOX21 = "/kaggle/input/tox21data/tox21.csv"
df = pd.read_csv(TOX21)

print('='*70)
print('📊 LOAD DATA')
print('='*70)
print(df.head())

label_cols = [
    "NR-AR", "NR-AR-LBD", "NR-AhR", "NR-Aromatase",
    "NR-ER", "NR-ER-LBD", "NR-PPAR-gamma",
    "SR-ARE", "SR-ATAD5", "SR-HSE", "SR-MMP", "SR-p53"
]

smiles_df = df["smiles"].tolist()
mol_ids_df = df["mol_id"].tolist()
labels_df = df[label_cols].values  # (n_samples, 12)

print(f"\n✅ SMILES: {len(smiles_df)} molecules")
print(f"✅ Labels shape: {labels_df.shape}")

# ============================================================================
# TẠO LABELS_CLEAN + MASK
# ============================================================================

labels_clean = np.nan_to_num(labels_df, nan=0.0)
mask = (~np.isnan(labels_df)).astype(np.float32)

print(f"✅ Labels_clean shape: {labels_clean.shape}")
print(f"✅ Mask shape: {mask.shape}")

# ============================================================================
# BUILD DATASET
# ============================================================================

dataset = []
failed = 0

print(f"\n{'='*70}")
print("🏗️  BUILD DATASET")
print(f"{'='*70}")

for i, smi in enumerate(smiles_df):
    # Convert SMILES → Mol
    try:
        mol = Chem.MolFromSmiles(smi, sanitize=False)
        Chem.SanitizeMol(mol)
    except:
        failed += 1
        continue
    
    if mol is None:
        failed += 1
        continue
    
    # ===== NODE FEATURES =====
    atom_features = []
    periodic_table = Chem.GetPeriodicTable()
    
    for atom in mol.GetAtoms():
        atomic_num = atom.GetAtomicNum()
        valence_electrons = periodic_table.GetNOuterElecs(atomic_num)
        features = [
            atomic_num,
            atom.GetTotalValence(),
            atom.GetTotalDegree(),
            int(atom.GetIsAromatic()),
            atom.GetFormalCharge(),
            valence_electrons,           
            atom.GetNumImplicitHs()      
        ]
        atom_features.append(features)
    
    x = torch.tensor(atom_features, dtype=torch.float)
    
    # ===== EDGE INDEX =====
    edge_index = []
    for bond in mol.GetBonds():
        a = bond.GetBeginAtomIdx()
        b = bond.GetEndAtomIdx()
        edge_index.append([a, b])
        edge_index.append([b, a])
    
    if len(edge_index) == 0:
        edge_index = torch.zeros((2, 0), dtype=torch.long)
    else:
        edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    
    # ===== LABELS (GRAPH-LEVEL!) =====
    y = torch.tensor(labels_clean[i], dtype=torch.float)        # (12,) ✅
    m = torch.tensor(mask[i], dtype=torch.float)                # (12,) ✅
    
    # ===== TẠO DATA OBJECT =====
    data = Data(
        x=x,
        edge_index=edge_index,
        y=y,
        mask=m,
        mol_id=mol_ids_df[i],
        num_nodes=x.shape[0]
    )
    
    dataset.append(data)

print(f"✅ Dataset created: {len(dataset)} molecules")
print(f"❌ Failed to parse: {failed} SMILES")

# Kiểm tra sample
sample = dataset[0]
print(f"\n📌 Sample Data Object:")
print(f"   x.shape: {sample.x.shape}")
print(f"   edge_index.shape: {sample.edge_index.shape}")
print(f"   y.shape: {sample.y.shape} ✅ (phải là (12,))")
print(f"   mask.shape: {sample.mask.shape} ✅ (phải là (12,))")
print(f"   num_nodes: {sample.num_nodes}")
print(f"   mol_id: {sample.mol_id}")

# ============================================================================
# SPLIT DATASET
# ============================================================================

print(f"\n{'='*70}")
print("📊 SPLIT DATASET")
print(f"{'='*70}")

random.seed(42)
random.shuffle(dataset)

total = len(dataset)
len_train = int(0.8 * total)
len_val = int(0.1 * total)
len_test = total - len_train - len_val

print(f"Tổng số mẫu: {total}")
print(f"Train: {len_train} (80%)")
print(f"Val:   {len_val} (10%)")
print(f"Test:  {len_test} (10%)")

train_dataset = dataset[:len_train]
val_dataset = dataset[len_train : len_train + len_val]
test_dataset = dataset[len_train + len_val :]

# ============================================================================
# VERIFY
# ============================================================================

print(f"\n{'='*70}")
print("✅ VERIFY DATASET")
print(f"{'='*70}")

print(f"\n📌 Train Sample:")
print(f"   x.shape: {train_dataset[0].x.shape}")
print(f"   y.shape: {train_dataset[0].y.shape} ✅")

print(f"\n📌 Val Sample:")
print(f"   x.shape: {val_dataset[0].x.shape}")
print(f"   y.shape: {val_dataset[0].y.shape} ✅")

print(f"\n📌 Test Sample:")
print(f"   x.shape: {test_dataset[0].x.shape}")
print(f"   y.shape: {test_dataset[0].y.shape} ✅")

# ============================================================================
# SAVE
# ============================================================================

print(f"\n{'='*70}")
print("💾 SAVE DATASET")
print(f"{'='*70}")

torch.save(train_dataset, "train.pt")
torch.save(val_dataset, "val.pt")
torch.save(test_dataset, "test.pt")

print("✅ train.pt saved")
print("✅ val.pt saved")
print("✅ test.pt saved")

print(f"\n{'='*70}")
print("✨ DONE!")
print(f"{'='*70}")

📊 LOAD DATA
   NR-AR  NR-AR-LBD  NR-AhR  NR-Aromatase  NR-ER  NR-ER-LBD  NR-PPAR-gamma  \
0    0.0        0.0     1.0           NaN    NaN        0.0            0.0   
1    0.0        0.0     0.0           0.0    0.0        0.0            0.0   
2    NaN        NaN     NaN           NaN    NaN        NaN            NaN   
3    0.0        0.0     0.0           0.0    0.0        0.0            0.0   
4    0.0        0.0     NaN           0.0    0.0        0.0            0.0   

   SR-ARE  SR-ATAD5  SR-HSE  SR-MMP  SR-p53   mol_id  \
0     1.0       0.0     0.0     0.0     0.0  TOX3021   
1     NaN       0.0     NaN     0.0     0.0  TOX3020   
2     0.0       NaN     0.0     NaN     NaN  TOX3024   
3     NaN       0.0     NaN     0.0     0.0  TOX3027   
4     0.0       0.0     0.0     NaN     0.0  TOX3028   

                                              smiles  
0                       CCOc1ccc2nc(S(N)(=O)=O)sc2c1  
1                          CCN1C(=O)NC(c2ccccc2)C1=O  
2  CC[C@]1(O)CC[C

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


✅ Dataset created: 8014 molecules
❌ Failed to parse: 0 SMILES

📌 Sample Data Object:
   x.shape: torch.Size([16, 7])
   edge_index.shape: torch.Size([2, 34])
   y.shape: torch.Size([12]) ✅ (phải là (12,))
   mask.shape: torch.Size([12]) ✅ (phải là (12,))
   num_nodes: 16
   mol_id: TOX3021

📊 SPLIT DATASET
Tổng số mẫu: 8014
Train: 6411 (80%)
Val:   801 (10%)
Test:  802 (10%)

✅ VERIFY DATASET

📌 Train Sample:
   x.shape: torch.Size([43, 7])
   y.shape: torch.Size([12]) ✅

📌 Val Sample:
   x.shape: torch.Size([31, 7])
   y.shape: torch.Size([12]) ✅

📌 Test Sample:
   x.shape: torch.Size([11, 7])
   y.shape: torch.Size([12]) ✅

💾 SAVE DATASET
✅ train.pt saved
✅ val.pt saved
✅ test.pt saved

✨ DONE!


In [20]:
train_dataset = torch.load("train.pt", weights_only=False)
val_dataset   = torch.load("val.pt", weights_only=False)
test_dataset  = torch.load("test.pt", weights_only=False)

In [21]:
data = train_dataset[0]
print(data)
print("Node feature matrix (x):")
print(data.x)
print("x shape:", data.x.shape)


Data(x=[43, 7], edge_index=[2, 98], y=[12], mask=[12], mol_id='TOX197', num_nodes=43)
Node feature matrix (x):
tensor([[ 6.,  4.,  4.,  0.,  0.,  4.,  3.],
        [ 6.,  4.,  4.,  0.,  0.,  4.,  1.],
        [ 6.,  4.,  4.,  0.,  0.,  4.,  3.],
        [ 6.,  4.,  4.,  0.,  0.,  4.,  2.],
        [ 6.,  4.,  4.,  0.,  0.,  4.,  0.],
        [ 6.,  4.,  3.,  0.,  0.,  4.,  0.],
        [ 8.,  2.,  1.,  0.,  0.,  6.,  0.],
        [ 7.,  3.,  3.,  0.,  0.,  5.,  0.],
        [ 6.,  4.,  4.,  0.,  0.,  4.,  2.],
        [ 6.,  4.,  4.,  0.,  0.,  4.,  2.],
        [ 6.,  4.,  4.,  0.,  0.,  4.,  2.],
        [ 6.,  4.,  4.,  0.,  0.,  4.,  0.],
        [ 6.,  4.,  4.,  0.,  0.,  4.,  0.],
        [ 8.,  2.,  2.,  0.,  0.,  6.,  1.],
        [ 8.,  2.,  2.,  0.,  0.,  6.,  0.],
        [ 6.,  4.,  4.,  0.,  0.,  4.,  0.],
        [ 7.,  3.,  3.,  0.,  0.,  5.,  1.],
        [ 6.,  4.,  3.,  0.,  0.,  4.,  0.],
        [ 8.,  2.,  1.,  0.,  0.,  6.,  0.],
        [ 6.,  4.,  4.,  0.,  0., 

In [22]:
def custom_collate_fn(data_list):
    """
    Custom collate function để fix batch.y shape
    
    VẤNĐỀ: 
    - PyG Batch nối batch.y thành 1D: (7, 12) → (84,)
    
    GIẢI PHÁP:
    - Reshape lại thành (num_graphs, 12)
    """
    
    # Tạo batch bình thường
    batch = Batch.from_data_list(data_list)
    
    # ===== FIX: RESHAPE batch.y =====
    num_graphs = batch.batch.max().item() + 1
    num_classes = 12
    
    # Nếu batch.y bị flatten, reshape lại
    if batch.y.dim() == 1 and batch.y.shape[0] == num_graphs * num_classes:
        batch.y = batch.y.view(num_graphs, num_classes)
    
    # Tương tự cho mask
    if hasattr(batch, 'mask') and batch.mask.dim() == 1 and batch.mask.shape[0] == num_graphs * num_classes:
        batch.mask = batch.mask.view(num_graphs, num_classes)
    
    return batch

# ============================================================================
# PHẦN 2: EXTRACT LABELS - ĐƠNGIẢN (vì batch.y đã (num_graphs, 12))
# ============================================================================

def extract_graph_labels(batch, num_classes, device):
    """
    Lấy graph-level labels từ batch
    
    Đơn giản vì batch.y đã được reshape thành (num_graphs, 12)
    """
    
    num_graphs = batch.batch.max().item() + 1
    
    # ===== batch.y đã là (num_graphs, 12) =====
    if batch.y.shape[0] == num_graphs and batch.y.shape[1] == num_classes:
        return batch.y
    
    # ===== Hoặc batch.y vẫn là 1D (fallback) =====
    elif batch.y.dim() == 1 and batch.y.shape[0] == num_graphs * num_classes:
        return batch.y.view(num_graphs, num_classes)
    
    else:
        print(f"⚠️ Unexpected batch.y shape: {batch.y.shape}")
        print(f"   Expected: ({num_graphs}, {num_classes}) or ({num_graphs * num_classes},)")
        return torch.zeros(num_graphs, num_classes, device=device)


## GAT MODEL

In [23]:
import torch
import torch.nn as nn
from torch_geometric.nn import GATConv, GATv2Conv, global_mean_pool, global_max_pool, global_add_pool
import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score, hamming_loss, accuracy_score

# ============================================================================
# 🚀 IMPROVED GAT ARCHITECTURE - OPTIMIZED FOR YOUR DATA
# ============================================================================

class GAT(nn.Module):
    """
    State-of-the-art GAT cho multi-label classification với imbalanced data:
    
    Features:
    - GATv2Conv (better than GAT)
    - Multiple pooling strategies (mean + max + sum)
    - Batch Normalization
    - Residual connections at graph level
    - Dropout regularization
    - Better weight initialization
    """
    def __init__(self, input_dim=7, hidden_dim=128, num_heads=8,
                 num_layers=3, embedding_dim=512, dropout=0.2, 
                 edge_dim=None, add_self_loops=True):
        super().__init__()
        
        self.num_layers = num_layers
        self.dropout = dropout
        
        # GAT Layers - using GATv2 (more expressive)
        self.convs = nn.ModuleList()
        self.batch_norms = nn.ModuleList()
        
        # Input layer
        self.convs.append(
            GATv2Conv(
                input_dim, 
                hidden_dim, 
                heads=num_heads,
                dropout=dropout,
                concat=True,
                add_self_loops=add_self_loops,
                edge_dim=edge_dim
            )
        )
        self.batch_norms.append(nn.BatchNorm1d(hidden_dim * num_heads))
        
        # Hidden layers
        for i in range(num_layers - 2):
            self.convs.append(
                GATv2Conv(
                    hidden_dim * num_heads,
                    hidden_dim,
                    heads=num_heads,
                    dropout=dropout,
                    concat=True,
                    add_self_loops=add_self_loops,
                    edge_dim=edge_dim
                )
            )
            self.batch_norms.append(nn.BatchNorm1d(hidden_dim * num_heads))
        
        # Final layer - average attention heads
        self.convs.append(
            GATv2Conv(
                hidden_dim * num_heads,
                hidden_dim,
                heads=num_heads,
                dropout=dropout,
                concat=False,  # Average instead of concat
                add_self_loops=add_self_loops,
                edge_dim=edge_dim
            )
        )
        self.batch_norms.append(nn.BatchNorm1d(hidden_dim))
        
        # Graph-level pooling projection
        # Concat mean + max + sum = 3 * hidden_dim
        pooling_dim = hidden_dim * 3
        
        # MLP for graph embedding
        self.graph_mlp = nn.Sequential(
            nn.Linear(pooling_dim, embedding_dim),
            nn.BatchNorm1d(embedding_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            
            nn.Linear(embedding_dim, embedding_dim),
            nn.BatchNorm1d(embedding_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout)
        )
        
        # Initialize weights
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        """Xavier initialization for better gradient flow"""
        if isinstance(module, nn.Linear):
            nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
        elif isinstance(module, nn.BatchNorm1d):
            nn.init.ones_(module.weight)
            nn.init.zeros_(module.bias)
    
    def forward(self, x, edge_index, batch, edge_attr=None):
        """
        Args:
            x: Node features [num_nodes, input_dim]
            edge_index: Edge indices [2, num_edges]
            batch: Batch assignment [num_nodes]
            edge_attr: Edge features [num_edges, edge_dim] (optional)
        """
        
        # Message passing
        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index, edge_attr=edge_attr)
            x = self.batch_norms[i](x)
            x = torch.relu(x)
            x = nn.functional.dropout(x, p=self.dropout, training=self.training)
        
        # Multiple pooling strategies
        x_mean = global_mean_pool(x, batch)
        x_max = global_max_pool(x, batch)
        x_sum = global_add_pool(x, batch)
        
        # Concatenate poolings
        graph_repr = torch.cat([x_mean, x_max, x_sum], dim=-1)
        
        # Graph-level MLP
        graph_embedding = self.graph_mlp(graph_repr)
        
        return graph_embedding


In [24]:
class MultiLabelClassifier(nn.Module):
    """
    Advanced classifier với temperature scaling và multiple hidden layers
    """
    def __init__(self, embedding_dim=512, num_classes=12, 
                 hidden_dim=256, dropout=0.3, temperature=1.0):
        super().__init__()
        
        self.classifier = nn.Sequential(
            nn.Linear(embedding_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout),
            
            nn.Linear(hidden_dim // 2, num_classes)
        )
        
        # Temperature parameter for calibration
        self.temperature = nn.Parameter(torch.ones(1) * temperature)
        
        # Initialize
        self.apply(self._init_weights)
    
    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
    
    def forward(self, x):
        logits = self.classifier(x)
        # Apply temperature scaling
        scaled_logits = logits / torch.clamp(self.temperature, min=0.5, max=2.0)
        return scaled_logits



In [25]:
# ============================================================================
# 🔥 ASYMMETRIC LOSS - BEST FOR IMBALANCED MULTI-LABEL

class AsymmetricLossOptimized(nn.Module):
    """
    Asymmetric Loss specifically designed for imbalanced multi-label
   
    """
    def __init__(self, gamma_neg=4, gamma_pos=1, clip=0.05, 
                 disable_torch_grad_focal_loss=True):
        super().__init__()
        self.gamma_neg = gamma_neg
        self.gamma_pos = gamma_pos
        self.clip = clip
        self.disable_torch_grad_focal_loss = disable_torch_grad_focal_loss
    
    def forward(self, x, y):
        """
        Args:
            x: logits [batch_size, num_classes]
            y: targets [batch_size, num_classes] (0 or 1)
        """
        # Probabilities
        x_sigmoid = torch.sigmoid(x)
        xs_pos = x_sigmoid
        xs_neg = 1 - x_sigmoid

        # Asymmetric Clipping
        if self.clip is not None and self.clip > 0:
            xs_neg = (xs_neg + self.clip).clamp(max=1)

        # Basic CE
        los_pos = y * torch.log(xs_pos.clamp(min=1e-8))
        los_neg = (1 - y) * torch.log(xs_neg.clamp(min=1e-8))
        loss = los_pos + los_neg

        # Asymmetric Focusing
        if self.gamma_neg > 0 or self.gamma_pos > 0:
            if self.disable_torch_grad_focal_loss:
                torch.set_grad_enabled(False)
            
            pt0 = xs_pos * y
            pt1 = xs_neg * (1 - y)
            pt = pt0 + pt1
            one_sided_gamma = self.gamma_pos * y + self.gamma_neg * (1 - y)
            one_sided_w = torch.pow(1 - pt, one_sided_gamma)
            
            if self.disable_torch_grad_focal_loss:
                torch.set_grad_enabled(True)
            
            loss *= one_sided_w

        return -loss.mean()



# Train

In [26]:
def _find_optimal_thresholds(all_probs, all_labels, num_classes):
    """Improved threshold finding using multiple strategies"""
    from sklearn.metrics import precision_recall_curve
    
    optimal_thresholds = []
    per_class_f1 = []
    
    for i in range(num_classes):
        probs = all_probs[:, i]
        labels = all_labels[:, i]
        
        # Skip if no positive samples
        if labels.sum() == 0:
            optimal_thresholds.append(0.5)
            per_class_f1.append(0.0)
            continue
            
        # Strategy 1: Precision-Recall curve (more robust)
        try:
            precision, recall, thresholds = precision_recall_curve(labels, probs)
            f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)
            best_idx = np.argmax(f1_scores[:-1])
            thresh_pr = thresholds[best_idx]
            f1_pr = f1_scores[best_idx]
        except:
            thresh_pr = 0.5
            f1_pr = 0.0
        
        # Strategy 2: Adaptive grid search around PR threshold
        mean_prob = probs.mean()
        pos_ratio = labels.mean()
        
        # Smart search range based on class characteristics
        if pos_ratio < 0.1:  # Rare class
            search_min = max(0.01, thresh_pr - 0.1, mean_prob * 0.3)
            search_max = min(0.3, thresh_pr + 0.1, mean_prob * 2.0)
            n_points = 25
        elif pos_ratio > 0.9:  # Common class
            search_min = max(0.7, thresh_pr - 0.1, mean_prob * 0.8)
            search_max = min(0.99, thresh_pr + 0.1, mean_prob * 1.2)
            n_points = 25
        else:
            search_min = max(0.05, thresh_pr - 0.15, mean_prob * 0.5)
            search_max = min(0.95, thresh_pr + 0.15, mean_prob * 1.5)
            n_points = 20
        
        # Fine-tune with grid search
        best_f1 = f1_pr
        best_thresh = thresh_pr
        
        for thresh in np.linspace(search_min, search_max, n_points):
            preds = (probs > thresh).astype(int)
            f1 = f1_score(labels, preds, zero_division=0)
            if f1 > best_f1:
                best_f1 = f1
                best_thresh = thresh
        
        # Ensure threshold is reasonable
        best_thresh = np.clip(best_thresh, 0.05, 0.95)
        
        optimal_thresholds.append(best_thresh)
        per_class_f1.append(best_f1)
    
    return optimal_thresholds, per_class_f1

In [32]:
def _ensure_shape_compatibility(y_true, logits, num_classes, device):
    """Ensure y_true has same shape as logits"""
    if y_true.shape != logits.shape:
        y_true = y_true.view(logits.shape[0], -1)
        if y_true.shape[1] < num_classes:
            pad = num_classes - y_true.shape[1]
            y_true = torch.cat([
                y_true,
                torch.zeros(y_true.shape[0], pad, device=device)
            ], dim=1)
        elif y_true.shape[1] > num_classes:
            y_true = y_true[:, :num_classes]
    return y_true

In [27]:
def train_gat(model, classifier, train_loader, val_loader, device,
                             num_classes=12, epochs=100, patience=20):
    """
    Improved training function with better threshold optimization and error handling
    """
    
    print("\n" + "="*80)
    print("📊 ANALYZING TRAINING DATA")
    print("="*80)
    
    # Calculate class statistics
    all_labels = []
    for batch in train_loader:
        y_true = extract_graph_labels(batch, num_classes, device)
        all_labels.append(y_true.cpu().numpy())
    
    all_labels = np.vstack(all_labels)
    pos_counts = all_labels.sum(axis=0)
    total_samples = len(all_labels)
    
    print("\n📈 Class Distribution:")
    print(f"{'Class':<8} {'Samples':<10} {'Ratio':<10} {'Imbalance'}")
    print("-" * 80)
    
    for i in range(num_classes):
        ratio = pos_counts[i] / total_samples
        imbalance = (total_samples - pos_counts[i]) / (pos_counts[i] + 1e-5)
        print(f"Class {i:<2d} {int(pos_counts[i]):<10} {ratio*100:<9.2f}% {imbalance:<.2f}:1")
    
    # Setup
    model.to(device)
    classifier.to(device)
    
    # Optimizer with different learning rates
    optimizer = torch.optim.AdamW([
        {'params': model.parameters(), 'lr': 0.001, 'weight_decay': 0.01},
        {'params': classifier.parameters(), 'lr': 0.002, 'weight_decay': 0.005}
    ])
    
    # Loss function
    criterion = AsymmetricLossOptimized(
        gamma_neg=4,
        gamma_pos=1,
        clip=0.05
    )
    
    # Learning rate scheduler
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=8, min_lr=1e-6
    )
    
    # Warmup scheduler
    warmup_scheduler = torch.optim.lr_scheduler.LinearLR(
        optimizer, start_factor=0.1, total_iters=5
    )
    
    best_f1 = 0.0
    best_epoch = 0
    patience_counter = 0
    
    print("\n" + "="*80)
    print("🚀 STARTING TRAINING")
    print("="*80)
    
    for epoch in range(epochs):
        # ===== TRAINING =====
        model.train()
        classifier.train()
        
        total_loss = 0.0
        train_probs_list = []
        
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            
            # Forward
            graph_embedding = model(batch.x, batch.edge_index, batch.batch)
            logits = classifier(graph_embedding)
            
            # Get labels
            y_true = extract_graph_labels(batch, num_classes, device)
            
            # Ensure shape compatibility
            y_true = _ensure_shape_compatibility(y_true, logits, num_classes, device)
            
            # Loss & backward
            loss = criterion(logits, y_true.float())
            loss.backward()
            
            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(
                list(model.parameters()) + list(classifier.parameters()),
                max_norm=1.0
            )
            
            optimizer.step()
            total_loss += loss.item()
            
            # Monitor probabilities
            with torch.no_grad():
                train_probs_list.append(torch.sigmoid(logits).mean().item())
        
        # Warmup for first 5 epochs
        if epoch < 5:
            warmup_scheduler.step()
        
        avg_train_loss = total_loss / len(train_loader)
        avg_train_prob = np.mean(train_probs_list)
        
        # ===== VALIDATION =====
        model.eval()
        classifier.eval()
        
        val_loss = 0.0
        all_probs = []
        all_labels = []
        
        with torch.no_grad():
            for batch in val_loader:
                batch = batch.to(device)
                
                graph_embedding = model(batch.x, batch.edge_index, batch.batch)
                logits = classifier(graph_embedding)
                probs = torch.sigmoid(logits)
                
                y_true = extract_graph_labels(batch, num_classes, device)
                y_true = _ensure_shape_compatibility(y_true, logits, num_classes, device)
                
                loss = criterion(logits, y_true.float())
                val_loss += loss.item()
                
                all_probs.append(probs.detach().cpu().numpy())
                all_labels.append(y_true.detach().cpu().numpy())
        
        all_probs = np.vstack(all_probs)
        all_labels = np.vstack(all_labels)
        
        # IMPROVED: Better threshold optimization
        optimal_thresholds, per_class_f1 = _find_optimal_thresholds(
            all_probs, all_labels, num_classes
        )
        
        # Apply thresholds
        all_preds = (all_probs > optimal_thresholds).astype(int)
        
        # Calculate metrics
        micro_f1 = f1_score(all_labels, all_preds, average='micro', zero_division=0)
        macro_f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
        micro_precision = precision_score(all_labels, all_preds, average='micro', zero_division=0)
        micro_recall = recall_score(all_labels, all_preds, average='micro', zero_division=0)
        hamming = hamming_loss(all_labels, all_preds)
        
        avg_val_loss = val_loss / len(val_loader)
        mean_val_prob = all_probs.mean()
        
        # Print progress
        if (epoch + 1) % 5 == 0 or epoch == 0:
            print(f"\n{'='*80}")
            print(f"Epoch {epoch+1}/{epochs}")
            print(f"{'='*80}")
            print(f"📉 Loss:        Train={avg_train_loss:.4f} | Val={avg_val_loss:.4f}")
            print(f"📊 Mean Prob:   Train={avg_train_prob:.4f} | Val={mean_val_prob:.4f}")
            print(f"🎯 Micro F1:    {micro_f1:.4f} (P={micro_precision:.4f}, R={micro_recall:.4f})")
            print(f"🎯 Macro F1:    {macro_f1:.4f}")
            print(f"🎯 Hamming:     {hamming:.4f}")
            
            # Show class statistics
            low_f1_classes = np.argsort(per_class_f1)[:3]
            print(f"\n⚠️  Lowest F1 Classes:")
            for i in low_f1_classes:
                pos_ratio = all_labels[:, i].mean()
                print(f"   Class {i}: F1={per_class_f1[i]:.3f}, "
                      f"Thresh={optimal_thresholds[i]:.3f}, "
                      f"PosRatio={pos_ratio:.3f}")
        
        # LR scheduling
        if epoch >= 5:
            scheduler.step(micro_f1)
            
            # Print current learning rate
            if (epoch + 1) % 10 == 0:
                current_lr = optimizer.param_groups[0]['lr']
                print(f"   Current LR: {current_lr:.6f}")
        
        # Save best model
        if micro_f1 > best_f1:
            best_f1 = micro_f1
            best_epoch = epoch
            patience_counter = 0
            
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'classifier_state_dict': classifier.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_f1': best_f1,
                'optimal_thresholds': optimal_thresholds,
                'mean_probability': mean_val_prob,
                'per_class_f1': per_class_f1,
                'all_probs_val': all_probs,  # For analysis
                'all_labels_val': all_labels
            }, 'best_model_improved.pt')
            
            print(f"\n✅ Saved best model!")
            print(f"   Micro F1: {best_f1:.4f}")
            print(f"   Mean Prob: {mean_val_prob:.4f}")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"\n⚠️  Early stopping triggered at epoch {epoch+1}")
                print(f"   Best F1 was {best_f1:.4f} at epoch {best_epoch+1}")
                break
    
    print("\n" + "="*80)
    print("✨ TRAINING COMPLETED")
    print("="*80)
    print(f"🏆 Best Micro F1:     {best_f1:.4f}")
    print(f"📍 Best Epoch:        {best_epoch+1}")
    print(f"📊 Final Mean Prob:   {mean_val_prob:.4f}")
    print("="*80 + "\n")
    
    # Load best model
    checkpoint = torch.load('best_model_improved.pt', weights_only=False)
    model.load_state_dict(checkpoint['model_state_dict'])
    classifier.load_state_dict(checkpoint['classifier_state_dict'])
    
    return model, classifier, checkpoint['optimal_thresholds']


In [28]:
from sklearn.metrics import (
    accuracy_score, f1_score, hamming_loss, jaccard_score,
    precision_score, recall_score, classification_report
)
def evaluate_multilabel(model, classifier, data_loader, device, 
                         num_classes=12, optimal_thresholds=None):
    """
    Fixed evaluation function - no more RuntimeError!
    """
    
    model.eval()
    classifier.eval()
    
    all_probs = []
    all_labels = []
    
    with torch.no_grad():
        for batch in data_loader:
            batch = batch.to(device)
            
            graph_embedding = model(batch.x, batch.edge_index, batch.batch)
            logits = classifier(graph_embedding)
            probs = torch.sigmoid(logits)
            
            y_true = extract_graph_labels(batch, num_classes, device)
            
            if y_true.shape != logits.shape:
                y_true = y_true.view(logits.shape[0], -1)
                if y_true.shape[1] < num_classes:
                    pad = num_classes - y_true.shape[1]
                    y_true = torch.cat([
                        y_true,
                        torch.zeros(y_true.shape[0], pad, device=device)
                    ], dim=1)
                elif y_true.shape[1] > num_classes:
                    y_true = y_true[:, :num_classes]
            
            # FIX: Add .detach()
            all_probs.append(probs.detach().cpu().numpy())
            all_labels.append(y_true.detach().cpu().numpy())
    
    all_probs = np.vstack(all_probs)
    all_labels = np.vstack(all_labels)
    
    # Apply thresholds
    if optimal_thresholds is None:
        optimal_thresholds = [0.05] * num_classes
    
    all_preds = np.zeros_like(all_probs)
    for i in range(num_classes):
        all_preds[:, i] = (all_probs[:, i] > optimal_thresholds[i]).astype(int)
    
    # Metrics
    from sklearn.metrics import accuracy_score
    micro_f1 = f1_score(all_labels, all_preds, average='micro', zero_division=0)
    macro_f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)
    micro_p = precision_score(all_labels, all_preds, average='micro', zero_division=0)
    micro_r = recall_score(all_labels, all_preds, average='micro', zero_division=0)
    hamming = hamming_loss(all_labels, all_preds)
    subset_acc = accuracy_score(all_labels, all_preds)
    
    per_class_f1 = f1_score(all_labels, all_preds, average=None, zero_division=0)
    per_class_p = precision_score(all_labels, all_preds, average=None, zero_division=0)
    per_class_r = recall_score(all_labels, all_preds, average=None, zero_division=0)
    
    print("\n" + "="*80)
    print("📊 FINAL EVALUATION RESULTS")
    print("="*80)
    print(f"\n🎯 Overall Metrics:")
    print(f"   Micro F1:        {micro_f1:.4f}")
    print(f"   Macro F1:        {macro_f1:.4f}")
    print(f"   Micro Precision: {micro_p:.4f}")
    print(f"   Micro Recall:    {micro_r:.4f}")
    print(f"   Hamming Loss:    {hamming:.4f}")
    print(f"   Subset Accuracy: {subset_acc:.4f}")
    
    print(f"\n📈 Per-Class Results:")
    print(f"{'Class':<8} {'Threshold':<12} {'F1':<10} {'Precision':<12} {'Recall':<10} {'MeanProb':<10}")
    print("-" * 80)
    
    class_names = [
        "NR-AR", "NR-AR-LBD", "NR-AhR", "NR-Aromatase",
        "NR-ER", "NR-ER-LBD", "NR-PPAR-γ",
        "SR-ARE", "SR-ATAD5", "SR-HSE", "SR-MMP", "SR-p53"
    ]
    
    for i in range(num_classes):
        name = class_names[i] if i < len(class_names) else f"Class{i}"
        print(f"{name:<8} {optimal_thresholds[i]:<12.3f} "
              f"{per_class_f1[i]:<10.3f} {per_class_p[i]:<12.3f} "
              f"{per_class_r[i]:<10.3f} {all_probs[:, i].mean():<10.3f}")
    
    print("="*80 + "\n")
    
    return {
        'micro_f1': micro_f1,
        'macro_f1': macro_f1,
        'predictions': all_preds,
        'probabilities': all_probs,
        'labels': all_labels
    }



# Embedding

In [29]:
from tqdm import tqdm

def extract_embeddings(model, classifier, data_loader, device, 
                       num_classes=12, save_path='embeddings.pt'):
    """
    Extract graph embeddings từ GAT model và lưu ra file .pt
    
    Args:
        model: Trained GAT model
        classifier: Trained classifier
        data_loader: DataLoader (train/val/test)
        device: torch device
        num_classes: Số lượng classes
        save_path: Đường dẫn file output (.pt)
    
    Returns:
        embeddings_dict: Dictionary chứa embeddings và metadata
    """
    
    print("\n" + "="*80)
    print("🔍 EXTRACTING GRAPH EMBEDDINGS")
    print("="*80)
    
    model.to(device)
    classifier.to(device)
    model.eval()
    classifier.eval()
    
    all_embeddings = []
    all_logits = []
    all_probs = []
    all_labels = []
    all_graph_ids = []
    
    with torch.no_grad():
        for batch_idx, batch in enumerate(tqdm(data_loader, desc="Processing batches")):
            batch = batch.to(device)
            
            # Extract embeddings từ GAT
            graph_embedding = model(batch.x, batch.edge_index, batch.batch)
            
            # Get logits và probabilities từ classifier
            logits = classifier(graph_embedding)
            probs = torch.sigmoid(logits)
            
            # Get labels
            y_true = extract_graph_labels(batch, num_classes, device)
            
            # Ensure shape compatibility
            if y_true.shape != logits.shape:
                y_true = y_true.view(logits.shape[0], -1)
                if y_true.shape[1] < num_classes:
                    pad = num_classes - y_true.shape[1]
                    y_true = torch.cat([
                        y_true,
                        torch.zeros(y_true.shape[0], pad, device=device)
                    ], dim=1)
                elif y_true.shape[1] > num_classes:
                    y_true = y_true[:, :num_classes]
            
            # Store embeddings
            all_embeddings.append(graph_embedding.cpu())
            all_logits.append(logits.cpu())
            all_probs.append(probs.cpu())
            all_labels.append(y_true.cpu())
            
            # Store graph IDs nếu có
            if hasattr(batch, 'graph_id'):
                all_graph_ids.extend(batch.graph_id.cpu().tolist())
            else:
                # Generate sequential IDs
                batch_size = graph_embedding.shape[0]
                start_id = batch_idx * data_loader.batch_size
                all_graph_ids.extend(range(start_id, start_id + batch_size))
    
    # Concatenate all batches
    embeddings = torch.cat(all_embeddings, dim=0)
    logits = torch.cat(all_logits, dim=0)
    probs = torch.cat(all_probs, dim=0)
    labels = torch.cat(all_labels, dim=0)
    
    print(f"\n✅ Extracted {embeddings.shape[0]} graph embeddings")
    print(f"   Embedding dimension: {embeddings.shape[1]}")
    print(f"   Number of classes: {num_classes}")
    
    # Create embeddings dictionary
    embeddings_dict = {
        'embeddings': embeddings,              # [N, embedding_dim]
        'logits': logits,                      # [N, num_classes]
        'probabilities': probs,                # [N, num_classes]
        'labels': labels,                      # [N, num_classes]
        'graph_ids': all_graph_ids,           # List of graph IDs
        'embedding_dim': embeddings.shape[1],
        'num_graphs': embeddings.shape[0],
        'num_classes': num_classes,
        'model_info': {
            'model_type': model.__class__.__name__,
            'classifier_type': classifier.__class__.__name__,
        }
    }
    
    # Save to file
    torch.save(embeddings_dict, save_path)
    print(f"\n💾 Saved embeddings to: {save_path}")
    print(f"   File size: {os.path.getsize(save_path) / (1024*1024):.2f} MB")
    
    # Print statistics
    print(f"\n📊 EMBEDDING STATISTICS:")
    print(f"   Mean: {embeddings.mean().item():.4f}")
    print(f"   Std:  {embeddings.std().item():.4f}")
    print(f"   Min:  {embeddings.min().item():.4f}")
    print(f"   Max:  {embeddings.max().item():.4f}")
    
    print(f"\n📊 PROBABILITY STATISTICS:")
    print(f"   Mean: {probs.mean().item():.4f}")
    print(f"   Std:  {probs.std().item():.4f}")
    
    print("="*80 + "\n")
    
    return embeddings_dict



In [30]:
def extract_embeddings_by_split(model, classifier, train_loader, val_loader, 
                                test_loader, device, num_classes=12, 
                                output_dir='embeddings'):
    """
    Extract embeddings cho tất cả các splits (train/val/test) và lưu riêng
    
    Args:
        model: Trained GAT model
        classifier: Trained classifier
        train_loader, val_loader, test_loader: DataLoaders
        device: torch device
        num_classes: Số lượng classes
        output_dir: Thư mục output
    
    Returns:
        Dictionary chứa paths đến các embedding files
    """
    import os
    os.makedirs(output_dir, exist_ok=True)
    
    paths = {}
    
    # Extract train embeddings
    if train_loader is not None:
        print("\n🔹 Extracting TRAIN embeddings...")
        train_path = os.path.join(output_dir, 'train_embeddings.pt')
        extract_embeddings(model, classifier, train_loader, device, 
                          num_classes, train_path)
        paths['train'] = train_path
    
    # Extract validation embeddings
    if val_loader is not None:
        print("\n🔹 Extracting VALIDATION embeddings...")
        val_path = os.path.join(output_dir, 'val_embeddings.pt')
        extract_embeddings(model, classifier, val_loader, device, 
                          num_classes, val_path)
        paths['val'] = val_path
    
    # Extract test embeddings
    if test_loader is not None:
        print("\n🔹 Extracting TEST embeddings...")
        test_path = os.path.join(output_dir, 'test_embeddings.pt')
        extract_embeddings(model, classifier, test_loader, device, 
                          num_classes, test_path)
        paths['test'] = test_path
    
    print("\n" + "="*80)
    print("✨ ALL EMBEDDINGS EXTRACTED SUCCESSFULLY")
    print("="*80)
    for split, path in paths.items():
        print(f"   {split.upper()}: {path}")
    print("="*80 + "\n")
    
    return paths


In [33]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"✅ Dataset: {len(dataset)} samples")
print(f"   Train: {len(train_dataset)} samples")
print(f"   Val:   {len(val_dataset)} samples")
print(f"   Test:  {len(test_dataset)} samples\n")

# ============================================================================
# SETUP DEVICE
# ============================================================================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"📱 Device: {device}\n")

# ============================================================================
# INITIALIZE MODELS (FIX: Xóa dòng classifier = nn.Linear thừa)
# ============================================================================
print("🔧 Initializing models...")

# 1. GAT Model
model = GAT(
    input_dim=7,          # số đặc trưng input của node
    hidden_dim=128,       # hidden dimension
    num_heads=8,          # số attention heads
    num_layers=3,         # số layer GAT
    embedding_dim=512,    # kích thước embedding cuối
    dropout=0.2
)

# 2. Classifier (FIX: Chỉ khởi tạo 1 lần)
classifier = MultiLabelClassifier(
    embedding_dim=512,
    num_classes=12,
    hidden_dim=256,
    dropout=0.3,
    temperature=1.0
)

print(f"✅ GAT parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"✅ Classifier parameters: {sum(p.numel() for p in classifier.parameters()):,}\n")

# ============================================================================
# TRAINING
# ============================================================================
print("🚀 Starting training...\n")

model, classifier, optimal_thresholds = train_gat(
    model=model,
    classifier=classifier,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    num_classes=12,
    epochs=200,
    patience=20
)

# ============================================================================
# EVALUATION ON TEST SET
# ============================================================================
print("\n📊 Evaluating on test set...")

test_results = evaluate_multilabel(
    model=model,
    classifier=classifier,
    data_loader=test_loader,
    device=device,
    num_classes=12,
    optimal_thresholds=optimal_thresholds
)

# Print final results
print("\n" + "="*80)
print("🏆 FINAL TEST RESULTS")
print("="*80)
print(f"Micro F1:       {test_results['micro_f1']:.4f}")
print(f"Macro F1:       {test_results['macro_f1']:.4f}")




print("="*80 + "\n")

# ============================================================================
# EXTRACT EMBEDDINGS (OPTIONAL)
# ============================================================================
print("🔍 Extracting embeddings from trained model...")

embeddings_paths = extract_embeddings_by_split(
    model=model,
    classifier=classifier,
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    device=device,
    num_classes=12,
    output_dir='embeddings'
)

/tmp/ipykernel_48/588936053.py:1: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
/tmp/ipykernel_48/588936053.py:2: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
/tmp/ipykernel_48/588936053.py:3: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


✅ Dataset: 8014 samples
   Train: 6411 samples
   Val:   801 samples
   Test:  802 samples

📱 Device: cuda

🔧 Initializing models...
✅ GAT parameters: 4,686,208
✅ Classifier parameters: 166,541

🚀 Starting training...


📊 ANALYZING TRAINING DATA

📈 Class Distribution:
Class    Samples    Ratio      Imbalance
--------------------------------------------------------------------------------
Class 0  255        3.98     % 24.14:1
Class 1  188        2.93     % 33.10:1
Class 2  643        10.03    % 8.97:1
Class 3  223        3.48     % 27.75:1
Class 4  644        10.05    % 8.95:1
Class 5  281        4.38     % 21.81:1
Class 6  154        2.40     % 40.63:1
Class 7  774        12.07    % 7.28:1
Class 8  228        3.56     % 27.12:1
Class 9  298        4.65     % 20.51:1
Class 10 755        11.78    % 7.49:1
Class 11 345        5.38     % 17.58:1

🚀 STARTING TRAINING

Epoch 1/200
📉 Loss:        Train=0.0886 | Val=0.0421
📊 Mean Prob:   Train=0.4334 | Val=0.4450
🎯 Micro F1:    0.2179 (P=0.14

/usr/local/lib/python3.11/dist-packages/torch_geometric/utils/_scatter.py:91: UserWarning: The usage of `scatter(reduce='max')` can be accelerated via the 'torch-scatter' package, but it was not found
  warnings.warn(



✅ Saved best model!
   Micro F1: 0.2292
   Mean Prob: 0.4592


/usr/local/lib/python3.11/dist-packages/torch_geometric/utils/_scatter.py:91: UserWarning: The usage of `scatter(reduce='max')` can be accelerated via the 'torch-scatter' package, but it was not found
  warnings.warn(


...


✅ Extracted 6411 graph embeddings
   Embedding dimension: 512
   Number of classes: 12

💾 Saved embeddings to: embeddings/train_embeddings.pt
   File size: 13.42 MB

📊 EMBEDDING STATISTICS:
   Mean: 0.2929
   Std:  0.4950
   Min:  0.0000
   Max:  21.8663

📊 PROBABILITY STATISTICS:
   Mean: 0.3699
   Std:  0.1538


🔹 Extracting VALIDATION embeddings...

🔍 EXTRACTING GRAPH EMBEDDINGS


Processing batches: 100%|██████████| 26/26 [00:00<00:00, 149.86it/s]



✅ Extracted 801 graph embeddings
   Embedding dimension: 512
   Number of classes: 12

💾 Saved embeddings to: embeddings/val_embeddings.pt
   File size: 1.68 MB

📊 EMBEDDING STATISTICS:
   Mean: 0.2857
   Std:  0.4648
   Min:  0.0000
   Max:  9.7805

📊 PROBABILITY STATISTICS:
   Mean: 0.3706
   Std:  0.1492


🔹 Extracting TEST embeddings...

🔍 EXTRACTING GRAPH EMBEDDINGS


Processing batches: 100%|██████████| 26/26 [00:00<00:00, 153.27it/s]


✅ Extracted 802 graph embeddings
   Embedding dimension: 512
   Number of classes: 12

💾 Saved embeddings to: embeddings/test_embeddings.pt
   File size: 1.68 MB

📊 EMBEDDING STATISTICS:
   Mean: 0.2921
   Std:  0.4867
   Min:  0.0000
   Max:  11.2754

📊 PROBABILITY STATISTICS:
   Mean: 0.3727
   Std:  0.1492


✨ ALL EMBEDDINGS EXTRACTED SUCCESSFULLY
   TRAIN: embeddings/train_embeddings.pt
   VAL: embeddings/val_embeddings.pt
   TEST: embeddings/test_embeddings.pt

